# TP chapitre 2 - Entraînez-vous : sélectionnez le nombre de voisins dans un kNN pour une régression 

I just re-use all that I have done :

In [1]:
data_path = "~/Documents/openclassroom/Fomation_ingenieur_ML/data/"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## sklearn module : 
from sklearn import model_selection 
from sklearn import preprocessing
from sklearn import neighbors


## Preprocess : 

In [2]:
df = pd.read_csv(data_path+'winequality-red.csv', sep=";")
## DESIGN AND SAMPLING : 
X = df.drop("quality", axis = 1, inplace = False)
y = df.quality
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7)

## STANDARDIZATION : 
my_normalizer = preprocessing.StandardScaler().fit(X_train)
X_train_std = pd.DataFrame(my_normalizer.transform(X_train), index = X_train.index, columns = X_train.columns)
X_test_std = pd.DataFrame(my_normalizer.transform(X_test), index = X_test.index, columns = X_test.columns)

## Cross Validation with sklearn :

In [3]:
## CROSS VALIDATION :
param_grid = {"n_neighbors" : np.arange(8,14,1)}
nb_folds = 5
CV_reg = model_selection.GridSearchCV(neighbors.KNeighborsRegressor(),
                                      param_grid, cv=nb_folds)
CV_reg.fit(X_train_std, y_train)

## KNN REGRESSOR WITH THE BEST CV n_neighbors
knn_reg = neighbors.KNeighborsRegressor(**CV_reg.best_params_)
knn_reg.fit(X_train_std, y_train)

## PREDICTION : 
y_pred = knn_reg.predict(X_test_std)

## Cross Validation without sklearn CVGridSearch :
#### Annexe functions : 

In [4]:
## EXTRACT X_train, X_test FROM X AND train_index :    
def extract_Xsplitted_data(X, train_index):    
#     train_index = X_train.index
    test_index = [(index not in train_index) for index in X.index]
    X_train = X.loc[train_index]
    X_test = X.loc[test_index]
    return(X_train, X_test)

## STANDARDIZATION : 
def normalize_from_split(X, train_index):
    X_train, X_test = extract_Xsplitted_data(X, train_index)
    
    my_normalizer = preprocessing.StandardScaler().fit(X_train)
    X_train_std = pd.DataFrame(my_normalizer.transform(X_train), index = X_train.index, columns = X_train.columns)
    X_test_std = pd.DataFrame(my_normalizer.transform(X_test), index = X_test.index, columns = X_test.columns)

    return(X_train_std, X_test_std)


In [1]:
data = X_train, y_train 
## SET ARGUMENTS/PARAMETERS :
my_meth = neighbors.KNeighborsRegressor
# NUMBER OF FOLDS :
cv = 5
param_grid = {"n_neighbors" : np.arange(2,15,1)}

def my_CV(data, param_grid, my_meth) :
    ## toutes les variables "locales" sont préfixées de 'CV_' 

    # MAP THE DICT OF LIST INTO LIST OF DICT :
    param_dirg = model_selection.ParameterGrid(param_grid)
    ## INITIALIZATION : 
    CV_X, CV_y = data 
    k = 1
    res = {} # dict of dict 
    for kwargs in param_dirg :
        params_set = "_".join(str(val) for val in kwargs.values())
        res[params_set]={}

    ## SET FOLDS : 
    kf = model_selection.StratifiedKFold(n_splits = 5)
    CV_split_iterator = kf.split(CV_X, y = CV_y)

    ### LOOP ON FOLDS :
    for CV_train_range_index, CV_test_range_index in CV_split_iterator : 
        train_index = CV_X.index[CV_train_range_index]
        test_index = CV_X.index[CV_test_range_index] 

        ## NORMALIZE :
        CV_X_train, CV_X_test = normalize_from_split(CV_X, train_index)
        ## GET y SPLIT : 
        CV_y_train, CV_y_test = y[train_index], y[test_index]

        fold_key = "fold"+str(k)
        k+=1
        ### LOOP ON PARAM NAMES (HERE ONLY 1)
        for kwargs in param_dirg :
            ## FIT KNN on X_train AND PREDICT ON X_test :
            CV_meth = my_meth(**kwargs)
            CV_meth.fit(CV_X_train,CV_y_train)

            ## SAVE : 
            y_pred = CV_meth.predict(CV_X_test)
            y_table = pd.DataFrame(np.matrix((y_pred, CV_y_test)).T, columns=["pred","real"])

            params_set = "_".join(str(val) for val in kwargs.values())
            res[params_set][fold_key] = y_table
    return(res)
res= my_CV(data, param_grid, my_meth)

NameError: name 'X_train' is not defined

In [18]:
from sklearn import metrics
def compute_MSE(table):
    return(sklearn.metrics.mean_squared_error(y_true = table.real, y_pred=table.pred))

def compute_dict_MSE(tables):
    mse_vect = []
    for key, value in tables.items():
        mse_vect.append(compute_MSE(value))
    return(np.array(mse_vect))

from sklearn import metrics
def compute_R2(table):
    return(metrics.r2_score(y_true = table.real, y_pred=table.pred))

def compute_dict_R2(tables):
    r2_vect = []
    for key, value in tables.items():
        r2_vect.append(compute_R2(value))
    return(np.array(r2_vect))

In [19]:
MSE_mean = []
MSE_std = []

for params_set in res.keys(): 
    dict_y_table = res[params_set]
    MSE = compute_dict_MSE(dict_y_table)
    MSE_mean.append(MSE.mean())
    MSE_std.append(MSE.std())

params = []
for kwargs in model_selection.ParameterGrid(param_grid) :
    params.append(kwargs)

CV_results_ = {"params": params , "mean_test_score":MSE_mean, "std_test_score":MSE_std}


## new syntax to iterate <3 <3 : 
iterator = zip(CV_results_["mean_test_score"], CV_results_["std_test_score"], CV_results_["params"])
for mean, std, params in iterator:
    print("MSE = %0.3f (+/-%0.3f) for %s" %(mean, 2*std, params))
    

MSE = 0.534 (+/-0.099) for {'n_neighbors': 2}
MSE = 0.505 (+/-0.099) for {'n_neighbors': 3}
MSE = 0.491 (+/-0.094) for {'n_neighbors': 4}
MSE = 0.480 (+/-0.096) for {'n_neighbors': 5}
MSE = 0.474 (+/-0.092) for {'n_neighbors': 6}
MSE = 0.465 (+/-0.085) for {'n_neighbors': 7}
MSE = 0.459 (+/-0.081) for {'n_neighbors': 8}
MSE = 0.457 (+/-0.071) for {'n_neighbors': 9}
MSE = 0.453 (+/-0.069) for {'n_neighbors': 10}
MSE = 0.455 (+/-0.071) for {'n_neighbors': 11}
MSE = 0.455 (+/-0.068) for {'n_neighbors': 12}
MSE = 0.453 (+/-0.060) for {'n_neighbors': 13}
MSE = 0.453 (+/-0.056) for {'n_neighbors': 14}


In [20]:
R2_mean = []
R2_std = []

for params_set in res.keys(): 
    dict_y_table = res[params_set]
    R2 = compute_dict_R2(dict_y_table)
    R2_mean.append(R2.mean())
    R2_std.append(R2.std())

params = []
for kwargs in model_selection.ParameterGrid(param_grid) :
    params.append(kwargs)

CV_results_ = {"params": params , "mean_test_score":R2_mean, "std_test_score":R2_std}


## new syntax to iterate <3 <3 : 
iterator = zip(CV_results_["mean_test_score"], CV_results_["std_test_score"], CV_results_["params"])
for mean, std, params in iterator:
    print("R2 = %0.3f (+/-%0.3f) for %s" %(mean, 2*std, params))
    

R2 = 0.205 (+/-0.144) for {'n_neighbors': 2}
R2 = 0.249 (+/-0.135) for {'n_neighbors': 3}
R2 = 0.270 (+/-0.124) for {'n_neighbors': 4}
R2 = 0.286 (+/-0.130) for {'n_neighbors': 5}
R2 = 0.296 (+/-0.119) for {'n_neighbors': 6}
R2 = 0.309 (+/-0.109) for {'n_neighbors': 7}
R2 = 0.317 (+/-0.104) for {'n_neighbors': 8}
R2 = 0.321 (+/-0.087) for {'n_neighbors': 9}
R2 = 0.327 (+/-0.081) for {'n_neighbors': 10}
R2 = 0.323 (+/-0.084) for {'n_neighbors': 11}
R2 = 0.323 (+/-0.081) for {'n_neighbors': 12}
R2 = 0.326 (+/-0.070) for {'n_neighbors': 13}
R2 = 0.327 (+/-0.065) for {'n_neighbors': 14}
